<a href="https://colab.research.google.com/github/khrir/TEIN_Gambiarras/blob/main/Algoritmo_gen%C3%A9tico/besouros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
primeiro passo é criar uma população inicial de besouros.
poderíamos usar valores booleanos, strings ou valores inteiros.
para o nosso problema, irei utilizar valores inteiros de 0 a 255.
'''
import numpy as np
from numpy.random import randint

populacao = [] # define uma lista que receberá a pseudo população gerada.
numero_geracoes = 300 
tamanho_populacao = 100 # a população possui 100 besouros.
fitness_populacao = [] # define uma lista que receberá a avaliação de cada besouro gerado.

In [2]:
'''
Essa função irá gerar uma população de besouros a cada vez que for chamada.
Será gerado valores inteiros de 0 até 255, 100 vezes.
O array gerado apresenta três colunas.

ex:

pop = gerar_populacao()
print(pop[0])

~ [122  44  99]
'''
def gerar_populacao():
    return np.random.randint(255, size=(tamanho_populacao, 3))

In [3]:
'''
Função que retorna a combinação rgb de um besouro,
a soma de todos os valores e qual a posição ocupa

ex:

print(arr)
print(sum)
print(value)

~ [122, 44, 99]
~ 265
~ 0
'''
def fitness_function(x, index):
    return [x[0], x[1], x[2]], x[0] + x[1] + x[2], index

In [4]:
'''
Essa função soma os valores das três colunas em cada uma das 100 linhas.
Desse modo, ela mostra a avaliação de cada indivíduo.
ex: 

fit = avaliacao_fitness_populacao(pop)
print(fit)

~ [265, 234, 411, 409, 371, ...]
'''
def avaliacao_fitness_populacao(populacao):
    for index in range(tamanho_populacao):
        fitness_populacao.append(fitness_function(populacao[index], index)[1])
    return fitness_populacao

In [5]:
def selecao(populacao, fitness_populacao, k=3):
    # primeira seleção aleatória por torneio
    selecao_ix = randint(0, len(populacao))
    for ix in randint(0, len(populacao), k-1):
        if fitness_populacao[ix] < fitness_populacao[selecao_ix]:
            selecao_ix = ix
    return populacao[selecao_ix]


In [6]:
def crossover(p1, p2):
    c1, c2 = p1.copy(), p2.copy()
    c1 = [p1[0], p2[1], p2[2]]
    c2 = [p2[0], p1[1], p1[2]]
    return [c1, c2]

## Modificações na questão prática

1. Obter a seleção de indivíduos da população implementando o método da roleta;
2. Obter a seleção de indivíduos da população implementando o método de ranking;
3. Obter os melhores besouros modificando a quantidade de geração do algoritmo para: 300, 600 e 1200 gerações;
4. Implementar uma função que realize a mutação de 1% dos indivíduos a cada geração.



In [7]:
# # Célula para testar funções
# pop = gerar_populacao()

# arr, sum_arr, count = fitness_function(pop[0], 0)

# fit_acc = avaliacao_fitness_populacao(pop)
# # print(len(fit_acc))
# # select = selecao(pop, fit_acc)
# # print(select)

## Roulette wheel method

1. Encontrar a soma todos os valores da avaliação (S)
2. Encontrar a normalização dos valores da avaliação (populacao_fitness/S)
3. Encontrar a avaliação acumulativa 
4. Gerar um número aleatório 
5. Selecionar o indivíduo com valor menor que a avaliação acumulativa. 

In [8]:
def roulette(populacao, fitness_populacao):
    
    s = sum(fitness_populacao) # Soma todos os valores fitness
    prop = fitness_populacao/s # descobre a proporção entre valores fitness e o acumulativo
    prop_360 = prop * 360 # transforma a proporção em um círculo: 360°

    prop_acc = np.zeros(tamanho_populacao) # inicia um array de mesmo tamanho da população
    a = 0
    for i in range(tamanho_populacao):
        a += prop_360[i]
        prop_acc[i] = a
    
    index_selecionado = 0
    lance = random.uniform(0,1) * 360.0
    # print('Lance', lance)
    for k in range(tamanho_populacao):
        if prop_acc[k] > lance:
            index_selecionado = k
    return populacao[index_selecionado]

In [9]:
# Célula para testar o metódo da seleção por roleta
# for i in range (tamanho_populacao):
# roule = roulette(pop, fit_acc)
# print(roule)

## Ranking method
1. Ordenar os valores de avaliação da população
2. Dê uma probabilidade para a seleção de cada indivíduo da população
3. Calcule a avaliação acumulativa
4. Inicie o metódo da roleta

-- O pi não foi implementado, só uma gambiarra

1. Ordenar a lista por rank.
2. Escolher o valor máximo e mínimo 1 <= Max <= 1.2 | Min = (2 - Max)
3. Pi = (Max - (Max-Min) * (rank-1) / (m-1)) * 1/m


In [10]:
import random 
def ranking(populacao, fitness_populacao):
      #Ordenando a avaliação do menor para o maior
      sorted_fitness = fitness_populacao.copy()
      sorted_fitness.sort()

      # Ordenando a população por avaliação do menor para o maior
      # Dessa forma, é possível organizar por ordem
      # A menor avaliação será o 1º na ordem, e assim por diante
      sorted_populacao = []
      i, j = 0, 0

      for i in range(tamanho_populacao):
          for j in range(tamanho_populacao):
              if fitness_populacao[j] < sorted_fitness[i]:
                  sorted_populacao.append(populacao[j])

      selecionado = roulette(sorted_populacao, sorted_fitness)
      return selecionado
      

In [11]:
# Célula para testar o metódo de ranking
# rank = ranking(pop, fit_acc)
# print(rank)

In [12]:
def algoritmo_genetico():
    # gera uma população
    populacao = gerar_populacao() 
    # print(populacao)

    # inicia a função de treinamento, que por padrão retorna os dados da primeira linha
    melhor_besouro, melhor_pontuacao, melhor_posicao = fitness_function(populacao[0], 0)

    for gen in range(numero_geracoes):
        fitness_populacao = avaliacao_fitness_populacao(populacao)
    
        # verifica o melhor besouro
        for i in range(tamanho_populacao):
            if fitness_populacao[i] < melhor_pontuacao:
                melhor_besouro, melhor_pontuacao, melhor_posicao = populacao[i], fitness_populacao[i], i
                print(melhor_besouro, "É o melhor besouro, na posição %d, com pontuacao %d." % (melhor_posicao, melhor_pontuacao))

        # selecionar os melhores da população e realizar nova geração
        selecionados = [ranking(populacao, fitness_populacao) for _ in range(tamanho_populacao)]

        filhos = list()

        for i in range(0, tamanho_populacao, 2):
            p1, p2 = selecionados[i], selecionados[i+1]
            for filho in crossover(p1, p2):
                filhos.append(filho)
        populacao = filhos
    print("Algoritmo Genético:")
    print(populacao)
    return [melhor_besouro, melhor_pontuacao]


In [13]:
melhor_besouro, melhor_pontuacao = algoritmo_genetico()
print('Melhor besouro: %s; Melhor pontuação: %d' % (melhor_besouro, melhor_pontuacao))

[ 90  22 141] É o melhor besouro, na posição 1, com pontuacao 253.
[120  10 113] É o melhor besouro, na posição 6, com pontuacao 243.
[ 18 159  28] É o melhor besouro, na posição 7, com pontuacao 205.
[32 71 90] É o melhor besouro, na posição 13, com pontuacao 193.
[  2  21 100] É o melhor besouro, na posição 18, com pontuacao 123.
[37 61  7] É o melhor besouro, na posição 91, com pontuacao 105.
Algoritmo Genético:
[[43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 90, 67], [43, 9